Для начала импортируем все нужные нам библиотеуи и модули

In [5]:
from keras.layers import Conv2D, UpSampling2D, InputLayer
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb
from skimage.io import imsave
import numpy as np
from google.colab import files
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt

И, далее, нам нужно загрузить изображение "стилевое"

In [6]:
upl = files.upload()
names = list(upl.keys())
img = Image.open(BytesIO(upl[names[0]]))

Saving img_style.jpg to img_style (2).jpg


Далее пишем функцию для первичной обработки зображения

In [7]:
def processed_image(img):
  image = img.resize( (256, 256), Image.BILINEAR)
  image = np.array(image, dtype=float)
  size = image.shape
  lab = rgb2lab(1.0/255*image)
  X, Y = lab[:,:,0], lab[:,:,1:]

  Y /= 128    # нормируем выходные значение в диапазон от -1 до 1
  X = X.reshape(1, size[0], size[1], 1)
  Y = Y.reshape(1, size[0], size[1], 2)
  return X, Y, size

Запускаем функцию со стилевым изобрадением

In [8]:
X, Y, size = processed_image(img)

Дальше нам нужно построить модель СНС

In [9]:
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))

Для примера обучим эту НС на одном изображении

In [10]:
model.compile(optimizer='adam', loss='mse')
model.fit(x=X, y=Y, batch_size=1, epochs=50)

Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 0.0711
Epoch 2/50
1/1 [==============================] - 1s 952ms/step - loss: 0.5082
Epoch 3/50
1/1 [==============================] - 1s 954ms/step - loss: 0.8648
Epoch 4/50
1/1 [==============================] - 1s 961ms/step - loss: 0.9267
Epoch 5/50
1/1 [==============================] - 1s 948ms/step - loss: 0.6956
Epoch 6/50
1/1 [==============================] - 1s 935ms/step - loss: 0.3886
Epoch 7/50
1/1 [==============================] - 1s 930ms/step - loss: 0.0650
Epoch 8/50
1/1 [==============================] - 1s 947ms/step - loss: 0.0221
Epoch 9/50
1/1 [==============================] - 1s 949ms/step - loss: 0.0190
Epoch 10/50
1/1 [==============================] - 1s 937ms/step - loss: 0.0186
Epoch 11/50
1/1 [==============================] - 1s 966ms/step - loss: 0.0184
Epoch 12/50
1/1 [==============================] - 1s 953ms/step - loss: 0.0183
Epoch 13/50
1/1 [==============================] - 1

Так же передаем "контекстное" изобрадение

In [ ]:
upl = files.upload()
names = list(upl.keys())
img = Image.open(BytesIO(upl[names[0]]))
X, Y, size = processed_image(img)

После обучения прогоним через сеть изображение в градациях серого

In [ ]:
output = model.predict(X)

Посмотрим на результат

In [ ]:
output *= 128
min_vals, max_vals = -128, 127
ab = np.clip(output[0], min_vals, max_vals)

cur = np.zeros((size[0], size[1], 3))
cur[:,:,0] = np.clip(X[0][:,:,0], 0, 100)
cur[:,:,1:] = ab
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.subplot(1, 2, 2)
plt.imshow(lab2rgb(cur))